# Interactive plotter for theoretical incoherent scatter power spectrum and ACF

In [1]:
import ipywidgets as widgets
from ipywidgets import interact_manual, fixed
import numpy
import pylab as plt

In [2]:
def plot_is_spec_and_acf(indict, ne, te, ti, frac_2, vi, tau_max, freq_max):
    """
    Plot incoherent scatter power spectrum and ACF given input plasma parameters.
    
    indict: Dictionary of input parameters
    ne: electron density, m^-3
    te: electron temperature, K
    ti: ion temperature, K
    frac_2: fraction of ion #2
    vi: line-of-sight velocity, m/s
    tau_max: maximum lag for ACF plot, sec
    freq_max: maximum frequency for spectral plot, Hz """
    
    import ismodel
    
    cspec = ismodel.ISspec(indict, 
                           emode=[1,1,1],  # On: Magnetic field, Coulomb collisions, BGK ion-neutral collisions
                           imode=[1,1,1],  # On: Magnetic field, Coulomb collisions, BGK ion-neutral collisions
                           Nfreq=127,      # number of points in power spectrum
                           czparams=(1e-6,2e5,100,10,1.0)) # Gordeyev integral parameters (leave unaltered)

    # adjust for desired parameters
    adjdict = {'ne':ne,
               'te':te,
               'ti':ti*numpy.ones(2),
               've':vi,
               'vi':vi*numpy.ones(2),
               'ni':numpy.array([1.0-frac_2,frac_2])}
    cspec.adjustParams(adjdict)

    # compute spectrum, ACF
    ff,spec,tau,acf = cspec.computeSpec()
    
    # plot it
    f, ax = plt.subplots(1,2,figsize=(12,6))
    
    # power spectrum vs frequency
    ax[0].plot(ff/1e3, spec, 'k')
    ax[0].set_xlim(-freq_max/1e3,freq_max/1e3)
    ax[0].set_xlabel('Frequency (kHz)')
    ax[0].set_ylabel('Power Spectrum')
    ax[0].set_title('Mi = %s AMU  %% = %s' % (str(indict['mi']), str(adjdict['ni'])))
    ax[0].grid()
    
    # Real, imag part of ACF vs lag (tau)
    # Only plot 1/4 of the positive half of the ACF;
    # zero lag is 1/2 way into the ACF array (starts with most negative lag)
    tau0 = int(acf.shape[0]/2)
    tau1 = tau0 + int(acf.shape[0]/4)
    ax[1].plot(tau[tau0:tau1]*1e6, acf[tau0:tau1].real, 'r', label='Real')
    ax[1].plot(tau[tau0:tau1]*1e6, acf[tau0:tau1].imag, 'b', label='Imag')
    ax[1].grid()
    ax[1].set_xlim(0,tau_max*1e6)
    ax[1].set_xlabel('Lag Time (usec)')
    ax[1].set_ylabel('ACF')
    ax[1].legend()
    plt.show()

### Please note:

The parameters in "indict" are **not** on sliders.  For the exercises, some of these parameters will need to be edited by hand.  Don't forget to execute the code block below again after you've edited before running the interactive portion!

In [3]:
# set fixed parameters
indict={'Nion': 2, # number of ions \
      'mi': numpy.array([16, 30]), # list of ion masses, AMU \
      'B': 5e-5, # backgrond magnetic field, Tesla \
      'f0': 440.2e6, # Millstone Hill radar frequency, Hz \
      'te': 1000.0, # electron temperature, K \
      'alpha': 60.0, # angle between look direction and magnetic field, deg (90 = perp to B) \
      'ne': 1e11, # electron density, m^-3 \
      'ti': numpy.array([1000.0, 1000.0]), # ion temperature, K \
      'ni': numpy.array([1.0, 0.0]), # list of ion fractions (sum must = 1) \
      've': 0.0, # electron velocity, m/s \
      'vi': numpy.array([0.0, 0.0]), # ion velocity, m/s \
      'nuen': 0.0, # electron-neutral collision frequency, Hz \
      'nuin': numpy.array([0.0, 0.0]) # ion-neutral collision frequency, Hz \
       }

### Run this block to make interactive plots with sliders.

In [5]:
p = interact_manual(plot_is_spec_and_acf, 
             indict=fixed(indict),
             te=widgets.FloatSlider(1000,min=250,max=3000,step=50),
             ti=widgets.FloatSlider(1000,min=250,max=3000,step=50),
             ne=widgets.FloatLogSlider(1e11,base=10,min=11,max=12,step=0.1),
             vi=widgets.FloatSlider(0.0,min=0.0,max=1000.0,step=50),
             frac_2=widgets.FloatSlider(0.0,min=0.0,max=1.0,step=0.1),
             tau_max=fixed(500e-6),
             freq_max=fixed(10e3))

interactive(children=(FloatLogSlider(value=100000000000.0, description='ne', max=12.0, min=11.0), FloatSlider(…

## Group exercises:

1. What is the effect of very large Te compared to Ti?  Can you relate the spectral shape and ACF to the principles in the “Signal Processing” lectures?  Examine the zero crossing of the ACF.  How does it relate to the spectral width?

2. Alter the notebook so you are using two ion species in the topside ionosphere: O+ (AMU=16) and H+ (AMU=1).  Examine the 100% O+ and 100% H+ cases.  Try to produce a parameter set using all the “knobs” for 100% H+ that looks spectrally (or in ACF) like 100% O+.  How can this behavior be related to the ion-acoustic dispersion relation?  What conclusion can you draw about determining ion composition and determining plasma temperature at the same time with an IS radar?

3. Use the default case of two ion species: O+ (AMU=16) and NO+ (AMU=30).  Examine 100% O+ and 100% NO+ cases.  Try to produce a NO+ only parameter set using all the “knobs” that looks spectrally like O+.  How different are the parameter sets?  What conclusion can you draw regarding the ease (or difficulty) of remote sensing of NO+ vs. remote sensing of O+?

4. Use default parameters but introduce a non-zero velocity.  Can you explain the characteristics of the resulting complex ACF shape as well as the power spectrum?

5. Alter the notebook so you are using two ion species in the topside ionosphere: O+ (AMU=16) and H+ (AMU=1).  Examine three cases: 100% O+, 100% H+, and 50% of each species.  What are the characteristics of each case in the spectral and ACF domains?  How does this result demonstrate collective plasma behavior?

6. Change the transmit frequency from Millstone Hill (440.2 MHz) to Jicamarca (49.92 MHz).  What effect does this have on the power spectrum?  On the ACF?  Can you draw a conclusion regarding the length of pulse one would use at each facility to measure the IS power spectrum?
